## 1. Define Global Constants

In [25]:
import numpy as np
import math
from timeit import default_timer as timer
universe_size =  999999937
# 999983 # 1103 #999999937



## 2. Construct Universal Hash Functions

In [3]:
class CarterWegmanHash:
    def __init__(self, table_size):
        self.table_size = table_size

        self.a = np.random.randint(1, universe_size, dtype=np.int64)
        self.b = np.random.randint(0, universe_size, dtype=np.int64)

    def hash(self, key):
        return ((self.a * key + self.b) % universe_size) % self.table_size
    
    def __call__(self, key):
        return self.hash(key)


class CarterWegmanDictNaive:
    # a static hash table, output size = c * input size
    def __init__(self, input, table_size):
        self.hash = CarterWegmanHash(table_size)
        self.table = [[] for _ in range(table_size)]
        for i in range(len(input)):
            self.table[self.hash(input[i])].append(input[i])

    def query(self, key):
        list = self.table[self.hash(key)]
        for i in range(len(list)):
            if list[i] == key:
                return True
        return False

class CarterWegmanDictPerfect:
    # a static hash table, output size = c * input size
    def __init__(self, input, table_size):
        self.hash = CarterWegmanHash(table_size)
        tempTable = [[] for _ in range(table_size)]
        self.table = [None for _ in range(table_size)]
        for i in range(len(input)):
            tempTable[self.hash(input[i])].append(input[i])
        for i in range(len(input)):
            idx = self.hash(input[i])
            list = tempTable[idx]
            if len(list) == 0:
                continue
            self.table[idx] = CarterWegmanDictNaive(list, len(list)*len(list))

    def query(self, key):
        t2 = self.table[self.hash(key)]
        if t2 == None:
            return False
        return t2.query(key)



### a. Performance of naive implementation

In [120]:

input = np.random.randint(0, universe_size, 50000)
c = 3
dict = CarterWegmanDictNaive(input, int(len(input)*c))
collision = 0
col_pairs = 0
worst = 0
for list in dict.table:
    if len(list) > 1:
        collision += len(list)
    col_pairs += len(list) * (len(list)-1) / 2
    worst = max(worst, len(list))

start = timer()
for i in range(len(input)):
    dict.query(input[i])
duration = timer() - start

print('mapping ratio:', c)
print('avg collision:', collision/len(dict.table))
print('max collision:', worst)
print('total colliding pairs:', col_pairs)
print('avg query time:', duration / len(input))

mapping ratio: 3
avg collision: 0.09249333333333333
max collision: 6
total colliding pairs: 8207.0
avg query time: 1.8863140000030398e-06


### b. Performance of 2-level bootstrapped implementation

In [121]:

input = np.random.randint(0, universe_size, 50000)
dict = CarterWegmanDictPerfect(input, int(len(input)))
collision = 0
col_pairs = 0
worst = 0
total_bins = 0

for t2 in dict.table:
    if t2 is not None:
        for list in t2.table:
            if len(list) > 1:
                collision += len(list)
            col_pairs += len(list) * (len(list)-1) / 2
            worst = max(worst, len(list))
            total_bins += 1

start = timer()
for i in range(len(input)):
    dict.query(input[i])
duration = timer() - start

print('mapping ratio:', (total_bins+len(dict.table)) / len(input))
print('avg collision:', collision/total_bins)
print('max collision:', worst)
print('total colliding pairs:', col_pairs)
print('avg query time:', duration / len(input))

mapping ratio: 2.99656
avg collision: 0.07232439796449894
max collision: 3
total colliding pairs: 3676.0
avg query time: 2.993975999997929e-06


## 3. Construct Graph-based Minimal Perfect Hash

In [257]:
class MininalPerfectHash:
    def __init__(self, input, v_size, r):
        self.r = r
        self.v_size = v_size
        self.power = 2**math.ceil(math.log2(self.r))
        self.n = len(input)

        for _ in range(1000):
            self.E = np.zeros((self.n, self.r), dtype=np.int64)
            self.F = [CarterWegmanHash(self.v_size-i) for i in range(1, self.r+1, 1)]
            for x_idx in range(self.n):
                x = input[x_idx]
                v = np.zeros(self.r, dtype=np.int64)
                v[0] = self.F[0](x)
                for i in range(1, self.r, 1):
                    v[i] = v[0] + self.F[i](x) + 1
                    if v[i] >= v[i-1]: v[i] += 1
                v %= self.v_size
                self.E[x_idx,:] = v
            

            self.first = -1 * np.ones(self.v_size, dtype=np.int64)
            self.next = -1 * np.ones(self.v_size * self.power, dtype=np.int64)
            self.prev = -1 * np.ones(self.v_size * self.power, dtype=np.int64)
            self.mu = 0

            for x_idx in range(self.n):
                self.insert(self.E[x_idx,:])

            self.del_e_idx = []
            self.deleted = np.zeros(self.n, dtype=bool)  # Tracks if an edge is deleted
            for v_idx in range(self.v_size):
                self.recursive_delete(v_idx)

            # if no cycle, then break
            if len(self.del_e_idx) == self.n:
                break
        
        print("Acyclitic Graph Constructed", _)

        del_e_r = self.del_e_idx[::-1]
        # print("del_e_r", del_e_r)
        self.h_e = [0] * self.n  # Initialize with the correct unique numbers for each edge
        self.g_v = [-1] * v_size  # Initialize with -1 for each vertex

        for i in range(self.n):
            e_idx = del_e_r[i]
            e = self.E[e_idx]
            self.h_e[i] = e_idx
            
            # Sum of g(v) for all vertices in the edge except for the unassigned ones
            sum_g_v = 0
            unassigned_vertices = []
            for ve in e:
                if self.g_v[ve] == -1:
                    unassigned_vertices.append(ve)
                else:
                    sum_g_v += self.g_v[ve]
            
            # If there is at least one unassigned vertex
            if unassigned_vertices:
                # Assign the calculated value to the first unassigned vertex
                # and set the remaining unassigned vertices to 0
                self.g_v[unassigned_vertices[0]] = (i - sum_g_v) % self.n  #DEBUG HERE
                for ve in unassigned_vertices[1:]:
                    self.g_v[ve] = 0  # Set the remaining unassigned vertices to 0
            else:
                print("Error: All vertices are already assigned")
                break
            # check if the hash is correct
            testhash = 0
            for ve in e:
                testhash += self.g_v[ve]
            if testhash % self.n != i: 
                print("Error: Hash is incorrect")
                break

    def hash(self, x):
        v = np.zeros(self.r, dtype=np.int64)
        v[0] = self.F[0](x)
        for i in range(1, self.r, 1):
            v[i] = v[0] + self.F[i](x) + 1
            if v[i] >= v[i-1]: v[i] += 1
        v %= self.v_size

        sum_g_v = 0
        for i in range(self.r):
            sum_g_v += self.g_v[v[i]]
        e = sum_g_v % self.n
        return self.h_e[e]

    def __call__(self, x):
        return self.hash(x)

    def insert(self, v):
        for i in range(self.r):
            vi = v[i]
            r_idx = self.power * self.mu + i
            if self.first[vi] >= 0: self.prev[self.first[vi]] = r_idx
            self.next[r_idx] = self.first[vi]
            self.prev[r_idx] = -1
            self.first[vi] = r_idx
        self.mu += 1

        # print(f"After inserting edge {self.mu-1} between {v}:")
        # connected_edges = set()
        # for i in range(self.r):
        #     fn = self.first[v[i]]
        #     while fn >= 0:
        #         connected_edges.add(fn // self.power)
        #         fn = self.next[fn]
        # print("Connected Edges:", connected_edges)
        
    def delete(self, e_idx):
        for i in range(self.r):
            vi = self.E[e_idx, i]
            r_idx = self.power * e_idx + i
            if self.prev[r_idx] < 0:
                self.first[vi] = self.next[r_idx]
            else:
                self.next[self.prev[r_idx]] = self.next[r_idx]
            if self.next[r_idx] >= 0:
                self.prev[self.next[r_idx]] = self.prev[r_idx]
                
    def recursive_delete(self, v_idx):
        if self.first[v_idx] >= 0 and self.next[self.first[v_idx]] < 0: # only one edge
            e_idx = self.first[v_idx] // self.power # edge index
            if not self.deleted[e_idx]:
                self.deleted[e_idx] = True
                self.del_e_idx.append(e_idx)
                self.delete(e_idx)
                for i in range(self.r):
                    self.recursive_delete(self.E[e_idx, i])

class MinimalPerfectDict:
    def __init__(self, input, v_size, r):
        self.table = [0] * len(input)
        self.hash = MininalPerfectHash(input, v_size, r)
        for i in range(len(input)):
            self.table[self.hash(input[i])] = input[i]

    def query(self, key):
        return self.table[self.hash(key)] == key

### Performance of the Minimal Perfect Hash

In [289]:
r = 3
c = 3
input = np.random.randint(0, universe_size, 50000)
dict = MinimalPerfectDict(input, int(c * len(input)), r)

start = timer()
for i in range(len(input)):
    dict.query(input[i])
duration = timer() - start

print('mapping ratio:', c)
print('avg query time:', duration / len(input))

Acyclitic Graph Constructed 0
mapping ratio: 3
avg query time: 1.0830357999075203e-05


### Show Order-Preserving Property

In [290]:
r = 4
c = 3
input = np.random.randint(0, universe_size, 500)
rand_idx = np.random.randint(0, len(input), 5)
dict = MinimalPerfectDict(input, int(c * len(input)), r)

for i in range(len(rand_idx)):
    key = input[rand_idx[i]]
    print("input index:", rand_idx[i], "hashed index:", dict.hash(key))

Acyclitic Graph Constructed 0
input index: 305 hashed index: 305
input index: 343 hashed index: 343
input index: 43 hashed index: 43
input index: 425 hashed index: 425
input index: 487 hashed index: 487
